## <center><span style="color:yellow">Treci domaci iz Digitalne obrade slike<span><center>
###  <span style="color:blue">Student       : Veljko Ilic<span>
    
###  <span style="color:blue">Br. indeksa   : 2017/0611<span> 
    
###  <span style="color:blue">Datum predaje : 3.12.2022>

#### Celija koda ispod poziva sve potrebne biblioteke koje se koriste u izradi zadataka

In [ ]:
from __future__ import print_function

%matplotlib inline
import scipy
from scipy import ndimage
from scipy import *
from pylab import *

plt.style.use('dark_background');
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import skimage
from skimage import *
from skimage import color
from skimage import exposure
from skimage import filters
from skimage import feature
from skimage import io
import os

import numpy as np

### Zadatak 1.1 <span style="color:yellow"><center>Implementacija Canny-jevog algoritma za detekciju ivica<center><span>

Pri realizacij Kanijevog algoritma prva stvar koja se proveri jeste da li su prosledjeni podaci ispravni ili ne i ispit odgovarajuce poruke.Zatim je primenjen Gausov filtar kako bi se ocistio sum i da se ostri prelazi ispeglaju. Nakon toga ono sto sam uradio jeste da sam definisao Sobelov gradijentni operator i njega primenio po ***x*** i ***y*** koordinati ulazne slike.Slika na kojoj cu da testiram jeste slika broj tri.Ispod se nalazi rezultat prolaska Sobelovim operatorom za razlicite vrednosti sigma.Ono sto moze da se vidi iz prilozenog jeste da se sto vise smanjujem ***sigma*** blurovanje slike je manje pa su samim tim i ivice brojeva i kazaljku ostrije odnosno sto je sigma manje vise detalja izlazi u prvi plan.Kako se ***sigma*** poveca ivice postaju masnije tj. razlivenije.

>sigma = 2:
    >>![clock3](sekvence/canny1.jpg)
    

>sigma = 1:
    >>![clock3](sekvence/canny2.jpg)
    
    
>sigma = 0.4:
    >>![clock3](sekvence/canny3.jpg)   


Sto se tice kvantizacije uglova na jedan od cetiri pravaca tu nemam sta posebno da naglasi sem da je to sekvenca ***if*** uslova.Kada se odredi u kom pravcu se prostire ivica samo ispitam da li je trenutni piksel najveceg inteziteta u tom pravcu ako jeste onda se sadrzava u suprotnom se postavalja na nuli. To se naravno vrsi na novoj matrici koja posle obrade potiskivanja ne-lokalnih maksimuma sadrzi ivice izdvojene sa slike ali su sada te ivice debljine jednog piksela. 
Ovde cu uporedno prikazati i kako izgleda kada se potisnu ne-lokalni maksimumi i kada se odbace ivice razdvoje na slabe i jake.Dakle to mi je samo jos jedan for petlja koja za izdvojene ivice koje su izmedju gornjeg i donjeg praga racuna da li u susedstvu ima bar jedan pikesl vrednosti 1 i ako da onda i njemu dodeljuje vrednost jedan.Evo i par primera za ***sigma=0.8***,i razlicito ***treshol_low***,***treshold_high**. Nije da primecujem neku znacajno veliku razliku za malu pormenu vrednosti viseg praga, mozda je po meni samo obod sata manje izreckan, ali svakako je bitno da je sada slika sastavljena samo od 0 i 1.To kazem jer moze se primetiti da je slika svetlija, a to iz iz razloga sto kada se racunaju magnituda spektra i ugao vrednosti su u float poin.    

>sigma = 0.8,treshold_low = 0.2, treshold_high=0.55:
    >>![clock3](sekvence/canny4.jpg)
    
    
>sigma = 0.8, treshold_low = 0.2, treshold_high=0.75:
    >>![clock3](sekvence/canny5.jpg)   


Vidzeti su zakomentarisani kao i prikaz rezultata, dakle funkcija pri svom radu nece prikazivati nikakve rezultate vec samo vraca detektovane ivice.

In [ ]:
def canny_edge_detector(img_in,sigma_,treshold_low,treshold_high):
    '''
    "Canny Edge Detector implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Ova funkcija za ulaznu sliku vrsi izdvajanje njenih ivica pomocu Kanijevog algoritma.Pored slike koja treba da bude
    obradjena funkciji se prosledjuju i standardan devijacija Gausovog suma, vrednost nizeg prada i vrednost viseg praga.
    
    Inputs
    img_in        - siva slika na kojoj treba detektovati ivice
    sigma         - standardna devijacija Gausovog suma
    treshold_low  - gornja vrednost praga za detekciju jakih ivica
    treshold_high - donja vrednost praga za odbacijvanje slabih ivica
    
    Outputs
    img_canny - izdvojene ivice sa ulazne slike debljine jedan piksel     
    
    Example
    img_edges = canny_edge_detector(img_in,0.2,0.2,0.3)
    '''
    
    #Evaluating input data
    if(treshold_low<0):
        print('treshold_low has to be positive number!');
        return False;
    if(treshold_high<0):
        print('treshold_high has to be positive number!');
        return False;
    if(treshold_high<treshold_low):
        print('treshold_high can not be less then treshold_low!');
        return False;
    if(sigma_<0):
        print('std has to be positive number!');
        return False;
    
    #Noise Reduction of input image with Gaussian filter,radius=3
    img_gauss = skimage.filters.gaussian(img_in, sigma=sigma_, mode='nearest', truncate=3/sigma_);
    
    #Sobel gradient operator
    Hx=np.array([[-1,-2,-1],[0,0,0],[1,2,1]]);
    Hy=np.transpose(Hx);

    Gx=scipy.ndimage.convolve(img_gauss, Hx, mode='nearest');
    Gy=scipy.ndimage.convolve(img_gauss, Hy, mode='nearest');
    
    #Angle and Magnitude
    mag = np.sqrt(np.square(Gx) + np.square(Gy));
    angle=np.rad2deg(np.arctan2(Gy,Gx));
    
    #figure, ax=subplots(nrows=1, ncols=4, figsize=(12,4), dpi=80);

    #ax[0].imshow(image, cmap='gray', vmin=0, vmax=1), ax[0].set_title('Originalna slika'), ax[0].axis('off')
    #ax[1].imshow(image, cmap='gray', vmin=0, vmax=1), ax[1].set_title('Blurovana slika'), ax[1].axis('off')

    #ax[2].imshow(Gx, cmap='gray', vmin=amin(Gx), vmax=amax(Gx)), ax[2].set_title('Horizontalne ivice -\nvertikalni gradijent')
    #ax[2].axis('off')

    #ax[3].imshow(Gy, cmap='gray', vmin=amin(Gy), vmax=amax(Gy)), ax[3].set_title('Vertikalne ivice -\nhorizontalni gradijent')
    #ax[3].axis('off')

    #matplotlib.pyplot.tight_layout()
    #plt.show();

    #figure, ax=subplots(nrows=1, ncols=2, figsize=(15,6), dpi=80);
    #ax[0].imshow(mag, cmap='gray', vmin=0, vmax=1), ax[0].set_title('Magnituda gradijenta'), ax[0].axis('off');
    #im = ax[1].imshow(angle, cmap='jet')
    #ax[1].set_title('Ugao gradijenta'), ax[1].axis('off');
    #figure.colorbar(im, ax=ax[1], fraction=0.04, pad=0.08);


    #Determening compas regions
    [height,width]= img_in.shape;
    img_non_max=np.zeros(img_in.shape);

    for i in range(1,height-1):
        for j in range(1,width-1):
            q=0;
            r=0;

            #Horizontal edge
            if((angle[i,j]>=-22.5 and angle[i,j]<=22.5) or (angle[i,j]>=157.5 and angle[i,j]<= -157.5)):
                q = mag[i-1,j]
                r = mag[i+1,j]
            #-45 degre edge    
            elif((angle[i,j]>22.5 and angle[i,j]<=67.5) or (angle[i,j]> -157.5 and angle[i,j]<= -112.5)):  
                q = mag[i-1,j-1]
                r = mag[i+1,j+1]
            #Vertical edge    
            elif((angle[i,j]>67.5 and angle[i,j]<=112.5) or (angle[i,j]> -112.5 and angle[i,j]<= -67.5)):  
                q = mag[i,j-1]
                r = mag[i,j+1]
            #45 degre edge    
            else:
                q = mag[i-1,j+1]
                r = mag[i+1,j-1]
             
            if(mag[i,j] > q and mag[i,j] > r):
                img_non_max[i,j] = mag[i,j];
                
    #figure, ax=subplots(nrows=1,ncols=2,figsize=(12,6), dpi=80);
    #ax[0].imshow(img_non_max, cmap='gray', vmin=0, vmax=1), ax[0].set_title('Potisnuti ne-lokalni maksimumi')
    #ax[0].axis('off');
                
    #Histerezis tresholding
    tr_low  = treshold_low*np.max(img_non_max)
    tr_high = treshold_high*np.max(img_non_max)   
    
    #New image with same size
    img_canny = np.zeros(img_in.shape);
    
    #Determ. weak and strong edges.Also zero edges. 
    strong_i, strong_j = np.where(img_non_max>tr_high);
    weak_i,   weak_j   = np.where((img_non_max>=tr_low) & (img_non_max<=tr_high));
    zeros_i,  zeros_j  = np.where(img_non_max<tr_low); 
    
    img_canny[strong_i,strong_j] = 1;
    
    #Wheater weak edge is conected to strong or it si not an edge at all.
    for i in range(0,weak_i.size-1):
        if(img_non_max[weak_i[i]-1][weak_j[i]]==1 or img_non_max[weak_i[i]+1][weak_j[i]]==1 or\
          img_non_max[weak_i[i]][weak_j[i]-1]==1 or img_non_max[weak_i[i]][weak_j[i]+1]==1 or\
          img_non_max[weak_i[i]-1][weak_j[i]-1]==1 or img_non_max[weak_i[i]+1][weak_j[i]-1]==1 or\
          img_non_max[weak_i[i]-1][weak_j[i]+1]==1 or img_non_max[weak_i[i]+1][weak_j[i]+1]==1):
            
            img_canny[weak_i[i]][weak_j[i]]=1;
            
    #Ploting result
    #ax[1].imshow(img_canny, cmap='gray', vmin=0, vmax=1), ax[1].set_title('Double Histerezis')   
     
       
    return img_canny;    
#data_dir = 'sekvence/clocks/';
#Ovo je kul stvar ovaj as_gray!!! ali mora da ide pre toga io.imread
#image = skimage.img_as_float((io.imread(os.path.join(data_dir, 'clock3.png'),as_gray=True)));
#img_canny = canny_edge_detector(image,5,0,0)            

In [ ]:
#interact(canny_edge_detector, img_in = fixed(image), sigma_ = (0,5,0.1), treshold_low=(0,0.8,0.02), treshold_high=(0,0.8,0.05));

### Zadatak 1.2 <span style="color:yellow"><center>Implementacija funkcije ***get_line_segments***<center><span>

Implementacija funkcije ***get_line_segments*** koristi pomocnu funkciju ***chechSurrounding*** koja proverava lokalno susedstvo piksela koji se trenutno obradjuje.Nacin na koji sam ja realizovao resenje jeste da kada se dobiju uglovi i normalno rastojanje $(\rho,\theta)$ pomocu ugradjene funkcije **transform_hough_line** ja mogu iz jednacine $\rho=y*sin(\theta)+x*cos(\theta)$ da izrazim $y$ ili $x$.Sada recimo izrazim $y=\frac{\rho-x*cos(\theta)}{sin(\theta)}$ i kako mogu da dobijem jako lako sirinu slike onda za svaku vrednost $x$ iz opsega od nula do sirina slike mogu da dobijem koliko iznosi $y$. Ovo mi je bitno jer ja sada znam koji pikseli pripadaju pravi koju je detektovala ugradjena funkcija. Dalje kako znam koji pikseli pripadaju detektovanim pravama ja mogu da prodjem kroz sliku i da uporedim da li se na pozicijama koje pripadaju dobijenom pravu postoje i pikseli koji imaju vrednost jedan, ako postoji barem jedan krece se detekcijom duzi na datoj pravoj. Kako su pozicije piksela celi brojevi onda se moze desiti da li zbog zaokruzivana da piksel nije bas na mestu na kom se ocekuje pa se zato koristi ugradjena funkcija ***checkSurrounding*** koja proverava okolinu koja se zadaje parametrom **tolerancy**.Ako je prekid veci od dozvoljene vrednosti parametrom **max_gaps** onda se ispita da li je linija detektovana manja od duzine zadate parametrom **line_size**, ako nije onda se koordinate pocetka i kraja vracaju kao povratna vrednost funkcije.

In [ ]:
def checkSurrounding(img_in,tolerancy,x,y):
    if(img_in[y][x]==1):
        img_in[y][x]=0;
        return True;
    else:
        x1 = 0;y1 = 0;
        if((x - tolerancy)>=0):
            x1 = x - tolerancy;
            
        x2 = x + tolerancy+1;
        if(x2>=img_in.shape[1]):
            x2 = img_in.shape[1]-1;
        
        if((y - tolerancy)>=0):
            y1 = y - tolerancy;
        
        y2 = y + tolerancy+1;
        if(y2>=img_in.shape[0]):
            y2 = img_in.shape[0]-1;
        
        for i in range(y1,y2):
            for j in range(x1,x2):
                if(img_in[i][j]==1):
                    img_in[i][j]=0;
                    return True;
        return False;        

def get_line_segments(img_edges,line,min_size,max_gaps,tolerancy):
    '''
    "Canny Edge Detector implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Ova funkcija za ulaznu sliku vrsi izdvajanje njenih ivica pomocu Kanijevog algoritma.Pored slike koja treba da bude
    obradjena funkciji se prosledjuju i standardan devijacija Gausovog suma, vrednost nizeg prada i vrednost viseg praga.
    
    Inputs
    img_edges – slika sa detektovanim ivičnim pikselima sa koje se detektuju linije
    line      – niz od dva elementa (theta, rho) kojim se zadaje pravac linije
    min_size  – minimalna dužina segmenta (u pikselima) da bi duž bila detektovana, sve
                linije koje su manje od ove veličine treba da budu ignorisane. Povratni argument ove
                funkcije su sve duži čija je dužina veća od ovog parametra
    max_gaps  – maksimalne veličine prekida (u pikselima) koje se mogu ignorisati prilikom
                detekcije duži
    tolerancy – radijus okoline u okviru koje se traže ivični pikseli
    
    Outputs
    line_segments - izdvojene ivice sa ulazne slike debljine jedan piksel     
    
    Example
    img_edges = canny_edge_detector(img_in,0.2,0.2,0.3)
    '''
    #Evaluating if input data is correct
    if(line.size==0 or line.size==1):
        print("Morate proslediti i theta i ro parametar koji odredjuju duz!");
        return False;
    if(min_size<=0):
        print("Minimalna duzina segmenta mora biti vece od nule!");
        return False;
    if(max_gaps<0):
        print("Maksimalna duzina prekida mora biti veca od nule!");
        return False;
    if(tolerancy<0):
        print("Maksimalna duzina prekida mora biti veca od nule!");
        return False;
    #Creating empty list of line segments end points
    line_segments = [];
    
    #x and y coordinates of line 
    x_ = np.arange(0,img_edges.shape[1]-1);
    y_ = (line[1] - x_*np.cos(line[0]))/np.sin(line[0])
    
    corr_val = np.where((y_>=0) & (y_<=img_edges.shape[0]))
    
    x = x_[corr_val];
    y = y_[corr_val].astype(int);
    
    if(int(np.rad2deg(line[0]))==0):
        x = np.zeros(img_edges.shape[0]-1);
        y = np.arange(0,img_edges.shape[0]-1);
        x += int(line[1])+1;
        x = x.astype(int)
    
    begin = -1; 
    end = -1;
    
    line_length = 0;
    gap_length = 0;
    
    for i in range(x.size-1):
        #Does corresponding pixel exists
        if(checkSurrounding(img_edges,tolerancy,x[i],y[i])):
            #If it's a first dot on a line    
            if(begin==-1):
                begin=i;
                continue;
            if(gap_length<max_gaps):
                line_length = np.sqrt((y[i]-y[begin])**2 + (x[i]-x[begin])**2);
                end = i;
                gap_length = 0;
            else:
                if(line_length>=min_size):
                    line_segments.append((x[begin],y[begin]))
                    line_segments.append((x[end],y[end]));
                line_length = 0;
                begin = -1;
                end = -1;
                gap_length = 0;
        else:
            if(begin!=-1):
                gap_length += 1; 
                       
    
    return np.asarray(line_segments);

#### <span style="color:yellow"><center>Implementacija funkcije ***extract_time***<center><span>

Ova funkcija jeste da treba da prima razlicite slike ali sam je napravio da prima samo sivu sliku, odnosno kada se ova funkcija poziva odmah se vrsi ucitavanje slike kao sive. Prolazi se kroz sve prave koje detektuje Hafova transforamcija i dalje se prosledjuje **get_line_segment** pomocu koje se izdvajaju kazaljke. Zelim da pnapomenem da  je meni ugao pod kojim se traze kazaljke od $-\pi$ do $\pi$ jer za casovniksa slike 1 u suprotno ne detektuje kazaljku za sate.Kako su se neke kazaljke na casovnicima duplirale kada sam povecao ovaj ugao onda sam morao da kada se pomocu ***get_line_segments*** da proverim da li sam vec pod slicnim uglom(tj sa odredjenim odstupanjem od 5 stepeni) vec detektovao kazaljku. Tako da je meni svaka kazaljka kada je iscrtavam detektovana samo sa jednom crvenom linijom.

Prvi parametar koji je bilo tesko izbrati jeste sigma, jer iako se vidi da je u delu gde je implementiran Kanije algoritam da sto je sigma manje ivice bivaju ostrije medjutim to ovde kod detekcije ivica nije najpametnije jer sto je ugao ostriji i nije ceo npr 22.5 stepeni kao na slici ispod kazaljka za sate ne moze biti detektovana.Mozda za vece sigma ivica broje izgledaju okrnjeno mejdjutim mi zelimo da detektujemo kazaljke a ne brojeve. I ovaj parametar je bilo najteze namestiti jer je trebalo naci sweet spot za sve casovnike. Parametar za minimalnu duzinu linije je 84 jer ako je ispod toga onda se ne moze detektovati kazaljka na casovniku 9.

>cloclk1, sigma = 0.4,
    >>![clock1](sekvence/clock1sigma=0.4.jpg)
    
>clokc1, sigma = 1.75,
    >>![clock1s](sekvence/clock1sigma=1.8.jpg)    

>sigma = 1.75, min_size=85,
    >>![clock9](sekvence/clk9min_size=85.jpg) 
    
Kada takodje tolerancy se ovde mora postaviti na 2 inace se ne detektuje kazaljka za sate.
>sigma = 1.75, min_size=84,
    >>![clock9s](sekvence/clk9min_size=84.jpg) 
    
Ovaj proces je bio mucan dok nisam dosao do ovih nekih vrednosti pomocu kojih program radi ispravno tj. ispravno detektuje sate i minute.

Dakle kada se detektuje duz ona se doda u niz koji sam nazvao ***data*** a gde se ubacuju ugao pod koji je kazaljka,region kome pripada i duzine iste.Ugao mi vraca ugradjena funkcija za Hafovu transformaciju, duzinu odredjujem kao Euklidsko rastojanje, a region se odredjuje na osnovu centra sata. Pored toga sam mora da tom centru sata dam ofset jer nije na svakoj slici tacno u centru i sada pomocu ovih parametara ja odredjujem koliko ima sati.Duza kazljka je za minute, kraca za sate. Ako se kazaljke nalaze u regionu tri ili 4 onda se ne broj sati, minuta dodaje 6 odnosno 30. Ako je detektovana samo jedna kazaljka onda se smara da su se preklopile. Evo par primera koji ilustruju ispravan rad programa.

>Clock  1 time
    >>![clock1time](sekvence/clk1time.jpg)
    
>Clock 5 time
    >>![clock5time](sekvence/clk5.time.jpg)    

>Clokc 6 time
    >>![clock6time](sekvence/clk6time.jpg) 

In [ ]:
def extract_time(img_in):
    '''
    "Extract Time implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Ova funkcija za ulaznu sliku poziva funkciju koja implementira Kanijev algoritam detekcije ivica i nakod toga se
    pomocu ugradjene funkcije za hafovu transforamciju dobijaju detektovane ivice. Dalje obrada jeste izdvajanje kazaljki 
    sa sata i zatim odredjivanje koliko je sati.
    
    Inputs
    img_in - slika sa koje je potrebno ocitati vreme
    
    Outputs
    hourse,minute - povratna vresnot su sati i munti     
    
    Example
    hours,minute - extract_time(img_in)
    '''
    #Canny edge detecting
    img_edges = canny_edge_detector(img_in,1.75,0.3,0.5);
    origin = np.array((0, img_in.shape[1]))
    
    #Haf transform for finding lines
    [out, angles, distances] = skimage.transform.hough_line(img_edges,np.linspace(-np.pi,np.pi,360))
    [intensity, peak_angles, peak_distances] = skimage.transform.hough_line_peaks(out, angles, distances, \
                                                                                threshold=None, num_peaks=9) 
    # plot detected lines
    #fix, axes = plt.subplots(1, 1, figsize=(20, 8))
    #axes.imshow(img_edges, cmap='gray')
    #axes.set_title('Casovnik sa detektovanim kazaljkama.')
    #axes.set_xlim(origin)
    #axes.set_ylim((img_in.shape[0], 0))
    #axes.axis('off');
    #new array to store xbegin,xend,ybegin,yend
    line_segments = np.zeros(4);
    #array to store hours,minute,second
    data = np.zeros((3,3))-1;
    k = 0;
    for _, angle,dist  in zip(intensity, peak_angles, peak_distances):
        line = np.array([angle,dist])
        line_segments =  get_line_segments(img_edges,line,min_size=84,max_gaps=1,tolerancy=2);
        #Do the thing only if line is detected
        if(line_segments.size!=0):
            #So everything is between 0 and pi                   
            angle = np.rad2deg(angle)
            if(np.rad2deg(angle)<0):
                angle += 180

            found = False;
            #check if there is same specification line already detected
            for i in range(3):
                if(np.abs(angle-data[i][0])<=5):
                    found=True;
                    break;
            #If there is no two same elements        
            if(not found):
                #coord of begining and end
                x_begin = line_segments[0][0]
                x_end   = line_segments[1][0]
                y_begin = line_segments[0][1]
                y_end   = line_segments[1][1]
                x_centre = img_edges.shape[1]//2;
                y_centre = img_edges.shape[0]//2;
                #region where line is detected
                region = 0;
                if(x_begin>x_centre-50 and x_begin<x_centre+50):
                    if(y_end<y_centre):
                        region = 1;
                    else:
                        region = 2;
                elif((x_end>x_centre-50) and (x_end<x_centre+50)):
                    if(y_begin<y_centre):
                        region = 4;
                    else:
                        region = 3;
                #array od (angle,region,length)
                data[k][0] = angle;
                data[k][1] = region;
                data[k][2] = np.sqrt((y_end-y_begin)**2 + (x_end-x_begin)**2);
                k +=1;
                
                #axes.plot((line_segments[0][0],line_segments[1][0]),(line_segments[0][1],line_segments[1][1]),'-r');
    
    hours_index = 0;
    minute_index = 1;
    if(data[1][0]==-1.0):
        minute_index = hours_index;
    else:
        if(data[0][2]>data[1][2]):
            hours_index = 1;
            minute_index = 0;
        if(data[2][0]!=-1):
            if(data[2][2]>data[minute_index][2]):
                minute_index = 2
            elif(data[2][2]<data[hours_index][2]):
                hours_index = 2;
    #Determening time
    hours  = data[hours_index][0]//30;       
    minute  = data[minute_index][0]//6; 
    if(data[minute_index][1]>2):
        minute += 30;
    if(data[hours_index][1]>2):
        hours +=6;
        
    if((data[hours_index][1]==1.0 or data[hours_index][1]==4) and (np.abs(data[hours_index][0]-180)<=5) ):
        hours = 12;
    if((data[minute_index][1]==1.0 or data[minute_index][1]==4) and (np.abs(data[minute_index][0]-180)<=2) ):
         minute = 0;
    #if(data[hours_index][0])
    if(hours==0):
        hours=12;
    return hours,minute;

data_dir = 'sekvence/clocks'

for i in range(1,10):
    name = 'clock';
    if(i==7 or i==8):
        name = name+str(i)+'.jpg';
    else:
        name = name+str(i)+'.png';
    
    image = skimage.img_as_float((io.imread(os.path.join(data_dir, name))));
    image_gray = skimage.img_as_float((io.imread(os.path.join(data_dir, name),as_gray=True)))
    hours,minute =  extract_time(image_gray);
    
    title = str(int(hours)) + ':' + str(int(minute))
    fig, ax = plt.subplots(figsize = [12,6],dpi=100);
    ax.imshow(image,cmap='jet');
    ax.set_title(title), ax.axis('off')
    
    

### Zadatak 2.1 <span style="color:yellow"><center>Implementacija funkcije ***coin_mask***<center><span>

U funkciji ***coin_mask*** se pre svega slika prevede u HSV kolor sistem iz razloga sto je pozadina upecatljivo zelena, a sa idejom da se pozadina na kojoj su novcici izdvoji na histogramu, a naravno ono sto nije pozadina hocu da proglasim da je novicic i tako kreiram masku koja se dalje salje funkciji za labeliranje regiona.Preko Hue ravni se dosta dobro moze izdvojiti pozadina.Sama implementacija nakon transfera slike u HSV kolor sistem takodje filtrira sliku filtrom 13X13 jer ako se dobro zagledate u sliku 9e bez tog filtra meni su novcici koji su previse osvetljeni u H ravni(predstavljeni crvenom bojom) prosarani tj, imaju na sebi i zeleno i crvene i narandzaste boje, a ja sam zeleo da kada detektujem neke regione oni budu uniformni nebitno sto je maska i za taj region sa flitrom 3X3 dobra.

>Slika 6
>>![coin_mask6](sekvence/coin_mask6.jpg)
  
>Slika 7
>>![coin_mask7](sekvence/coin_mask7.jpg)


>Slika 9e
>>![coin_mask9er](sekvence/coin_mask9error.jpg)

>Slika 9
>>![coin_mask9](sekvence/coin_mask9.jpg)

In [ ]:
def coin_mask(img_in):
    '''
    "Coin Mask implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Funkcija koja na ocnovu ulazne slike pravi binarznu masku i to salje na izlaz.Fukcija detektuje koji su pikseli pozadine
    ostalo proglasava za regione od interesa za dalju obradu.Detekcija pozadine se vrsi u HSV kolor sistemu.
    
    Inputs
    img_in- Ulazna slika od koje treba napraviti binarnu masku
    
    Outputs
    img_bin - Binarizovana slika gde su detektovani regioni oznaceni sa 1 a pozadina sa 0.     
    
    Example
    img_bin = coin_mask(img_in)
    '''
    #Creating output matrix same saze as input image
    img_bin = np.zeros(img_in.shape,dtype=uint8)
    
    #transfer to HSV
    imgHSV = color.rgb2hsv(img_in);
    imgH = scipy.ndimage.median_filter(imgHSV[:,:,0], size=(13,13), mode='reflect')
    #Ploting results
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15,5), dpi=80)
    ax[0].imshow(image,cmap='jet'),ax[0].set_title('Originalna slika'), ax[0].axis('off')
    ax[1].imshow(imgH,cmap='jet'),ax[1].set_title('H ravan ulazne slike'), ax[1].axis('off')
    
    #Otsu method for threshold value 
    thresh1 = skimage.filters.threshold_otsu(imgH.flatten())
    thresh2 = 0.55;
    
    #Logical indexing 
    img_bin = ((imgH < thresh1) | (imgH>thresh2))
    ax[2].imshow(img_bin,cmap='gray'),ax[2].set_title('Maska izdvojena sa H ravni'), ax[2].axis('off')
    #Ploting histogram with threshold values
    #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,4), dpi=80)
    ax[3].hist(imgH.flatten(), bins=256, color='b'), ax[3].set_facecolor('white');
    ax[3].axvline(x=thresh1, color='r', linestyle='-');
    ax[3].axvline(x=thresh2, color='r', linestyle='-');
    ax[3].set_title('Histogram H ravni ulazne slika')
    tight_layout()
    plt.show()
    
    return img_bin.astype(int)

In [ ]:
data_dir = 'sekvence/coins/';

name = 'coins'
name += str(9)+'.jpg';

image = skimage.img_as_float((io.imread(os.path.join(data_dir, name))));

img_bin = coin_mask(image);


### Zadatak 2.2 <span style="color:yellow"><center>Implementacija funkcije ***bw_label***<center><span>

Sto se tice labeliranja regiona na kojima su novcici moja funkcija koristi jos dve pomocne funkcije ***conect8*** i ***updateEquivalencyList***.

Pri prvom prolasku kroz sliku ispituje se piksel po piksel, ako piksle ima vrednost nula onda se njemu dodeljuje labela 1.Ako piksel ima vrednost jedan onda se njemu dodeli nova labela ako su svi okolni pikseli 0 odnosno pikseli pozadine.
Okruzenje jednog piksela se odredjuje pomocu funkcije ***conect8*** koja ako su svi pikseli okruznja8 jednaki 0 vraca vr. Flase i dodoeljuje se nova labela. Ukoliko u okruznju postoje piskeli koji nisu 0 onda se ispita da li su njihove labele razlicite ako nisu razlicite onda se dodeli ta ista labela piksela iz okruzenja pikselu koji se obradjuje.

Ako postoji min dve razlicite labele onda se poziva funkcija ***updateEquivalencyList*** koja update-uje labele koje se nalaze u list1 i list2. Update-ovanje labele se vrsi tako sto ako je potrebno spojiti a->b i taj par ne postoji u list1(gde treba ocekivati a), a u list2(treba ocekivat b) onda se dodatno ispita i da li mozda u list2 postoji a? Zasto se ispituje da li u list2 postoji a? Recimo da treba spojiti a->b, a da u list1 postoji c kojem je par a u list2 onda je potrebno spojiti i c i b. Kako bih objasnio ovo na primerima ja sam osmislio moju dosta manju sliku na kojoj sam pre svega i testirao svoj kod. Par primera:

>Slika 6
>>![demo](sekvence/modelreg.jpg)
  



In [ ]:
def conect8(bw_img,i,j):
    '''
    "Connect8 implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Ovo je pomocna funkcija koja vrsi proveru da li se u susedstvu 8 nalaze pikseli koji nisu pozadina i te
    izdvojene piksele vraca nazad.Ako je nasla piksele koji nisu pozadina found=True u suprotno je False.
    
    Inputs
    bw_img   - matrica koja sadrizi labele dodeljenje do trenutnog piklse (i,j)    
    (i,j)    - koordinate trenutnog piksela koji se obradjuje
    
    Outputs
    found    - logicki bool tip
    neighbor - niz od vrednosti piksela iz okruzenja koji nisu pozadinksi pikseli
    
    Example
    found,neighborp = conect8(bw_img,i,j)
    '''
    neighbor = np.ones(4,dtype=np.uint16);  
    if((j-1)>=0):
        neighbor[0]  = bw_img[i][j-1];      #west pixel
        if((i-1)>=0):                       #
            neighbor[1] = bw_img[i-1][j-1]; #north-west pixel
    if((i-1)>=0):                           #
        neighbor[2] = bw_img[i-1][j];       #north pixel  
        if((j+1)<bw_img.shape[1]):          #
            neighbor[3] = bw_img[i-1][j+1]; #north-east pixel
    #Sorting ascending
    neighbor.sort();  
    #Find if all sorounding pixels are backgorund
    y = np.count_nonzero(neighbor==1);      
    
    found = False;
    if(y!=4):
        found = True;
    
    return found,neighbor[y:4];

def updateEquivalencyList(neighbor,list1,list2):
    '''
    "Updating Equivalency List implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Funkcija koja vrsi update liste ekvialenija izmedju labela koje se dodeljenje odredjenim pikselima.Ova funkcija se 
    poziva samo onda kada u 8 okruzenju ima piksela koji nizu oznaceni sa 1 tj. nisu pozadina i kada su pikseli iz okruzenja
    razliciti jedni od drugih. Kao povratnu vrednost salje update-ovane liste ekvivalencija.
    
    Inputs
    neighbor  - pikseli iz okruzenja 8  
    list1     - list1 predstavalja na koju se vrednost labele iz list2 slikaju
    list2     - labele koje treba zameniti sa labelama iz list1
    
    Outputs
    list1    - update-ovana lista1
    list2    - update-ovana lista2 
    
    Example
    list1, list2 = updateEquivalencyList(neighbor,list1,list2)
    '''
    #Go through array of pixels in neighbor.
    for i in range(0,neighbor.size):
        #if list1 is empty add new pair od labels in list1 and list2
        if(len(list1)==0):
            list1.append(neighbor[0]);
            list2.append(neighbor[1]);
        else:
            #logical ooperators
            found = False;
            found_l1 = False;
            index = 0;
            found_l2 = False;
            #examining if pair of labels alreay exists
            for j in range(1,neighbor.size):
                for i in range(0,len(list1)):
                    if(list1[i]==neighbor[0] and list2[i]==neighbor[j]):
                        found_l1 = True;
                        break;
                    #if there is no pair a->b, but there is b->c then conect a->c
                    if(list2[i]==neighbor[0]):
                        found_l2 = True;
                        index = i;
                        break;
                #if pair doesn't exist than add it
                if((not found_l1) and found_l2): 
                    for i in range(0,len(list1)):
                        if(list1[i]==list1[index] and list2[i]==neighbor[j]):
                            found = True;
                    if(not found):
                        list1.append(list1[index]);
                        list2.append(neighbor[j]);
                elif(not found_l1):
                    list1.append(neighbor[0]);
                    list2.append(neighbor[j]);
                found    = False;
                found_l1 = False;
                found_l2 = False;
    return list1,list2;

def bw_label(img_bin):
    '''
    "Connected-component labeling implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Funkcija koja vrsi spajanje povezanih regiona u formaciji 8, dakle svi ivicni pikseli i oni dijagonalni se ispituju.
    Funkciji mora biti prosledjena binazirovana slika.Odmah na pocetku ako je ulazna slika prosla proveru da li su parametri 
    dobri kreira se nova matrica u koju se smestaju labele.Pri prvom prolasku kroz sliku se za svaki piksel ispituje da li je 
    pozadina ili ne ako nije pozadina ona se taj piksel labelira.Nakon labeliranja svih piksela prolazi se i drugi put da se 
    iz tabele ekvivalencija izvrsi spajanje regiona. Nova matrica regiona se salje na izlaz. 
    Inputs
    img_bin = binarizovana ulazna slika gde je 0 pozadina, a 1 region od interesa
    
    Outputs
    bw_img - matrica koja sadrzi labele spojenih regiona    
    
    Example
    bw_img = bw_label(img_bin)
    '''
    [height, width] = img_bin.shape;
    if(height==0 or width==0):
        print("Ovoj funkciji se prosledjuje slika kao matrica i mora imate ne-nultu visinu i sirinu!");
        return False;
    if(np.unique(img_bin).size!=2):
        print("Niste prosledili binarizovanu sliku.");
        return False;
    
    #Creating new matrix like input image to store labels,all background labels are one
    #initialy set all to 1 but will be changed in for loops
    bw_img = np.zeros(img_bin.shape,dtype=np.uint16)-1;
    
    #new label starts from 5 because background label is 1, increment by one if 8 neighbor is all zeros   
    label = 2;
    #list's of labels for conected regions
    list1 = [];
    list2 = [];
    #Going through image matrix and labeling conected components
    for i in range(0,height):
        for j in range(0,width):
            #If pixel is zero than it is backgound and label is 1
            if(img_bin[i][j]==0):
                bw_img[i][j] = 1;
            #If pixel is 1 determening label
            else:
                #Finding 8 connected pixels on image, found is True if at least one neighb. pixel is not 1
                found ,neighbor = conect8(bw_img,i,j);
                if(found):
                    #If all sorounding pixels are same  than label is copied
                    result = np.all(neighbor == neighbor[0])
                    bw_img[i][j] = neighbor[0];
                    #Else if at least one pixel is diferent update Eq list's
                    if(result == False):
                        updateEquivalencyList(neighbor,list1,list2);
                else:
                    #If all neigh. pixel's are one than create new label
                    bw_img[i][j] = label;
                    label += 1;
                    
    #Second pass through image to connect component labels
    #bw[:] = bw_img[:]
    for i in range(0,height):
        for j in range(0,width):
            if(bw_img[i][j]!=1):
                if(list2.count(bw_img[i][j])):
                    index = list2.index(bw_img[i][j]);
                    bw_img[i][j] = list1[index];
    #print("Liste ekvivalencija")
    #print('list1')
    #print(list1)
    #print('list2')
    #print(list2)
    return bw_img;

In [ ]:
#img_bin = np.array([[0,0,1,0,0,0,0,0,0,0,1,0,0,0],
                    [1,1,1,1,0,0,0,0,1,1,1,1,1,0],
                    [0,0,1,0,0,0,0,0,0,0,1,0,0,0],
                    [0,0,0,0,1,1,0,0,0,0,1,0,0,0],
                    [0,1,0,0,0,0,1,0,1,1,1,0,0,0],
                    [0,0,0,0,1,1,0,0,0,0,0,0,0,0],
                    [0,0,1,1,0,1,0,0,0,0,0,0,0,0]]);

#img_bin = coin_mask(image);

#bw,img_out = bw_label(img_bin);
#print('Matrica regiona pre povezivanja')
#print(bw)
#fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,4),dpi=100);
#ax[0].imshow(bw,cmap='jet'),ax[0].set_title('Slika pre drugog prolaska'),ax[0].axis('off');
#ax[1].imshow(img_out,cmap='jet'),ax[1].set_title('Slika nakon povezivanja regiona'),ax[1].axis('off');

### Zadatak 2.3 <span style="color:yellow"><center>Implementacija funkcije ***coin_classification***<center><span>   

<center>Tabel koja prikazuje broj novcica po klasama, odnosno da program pri klasifikaciji ne gresi u odnosu na stvarno stanje <center>

|Slika              |Klasa 1            |Klasa 2               |Ukupno            |
|-------------------|-------------------|----------------------|------------------|
|      coins1       |        12         |        34            |       46         |
|      coins2       |         4         |        19            |       23         |
|      coins3       |        10         |         7            |       17         |
|      coins4       |         7         |         4            |       11         |
|      coins5       |        12         |        22            |       34         |
|      coins6       |        11         |        21            |       32         |
|      coins7       |        12         |        34            |       46         |
|      coins8       |        12         |        34            |       46         |
|      coins9       |        12         |        23            |       35         |

In [ ]:
def coin_classification(img_in):
    '''
    "Coin classification implementation"
    __Author__--Veljko Ilic, veljkoilic86@gmail.com
    
    Funkcija za ulaznu sliku na kojoj bi trebalo da budu novcici obradjuje tako da pre svega detektuje novcice i od 
    toga pravi binarnu masku pozivajuci fukciju coin_maks(img_in). Nakon sto se maska napravi poziva se funkcija poziva 
    fukciju za labeliranje povezanih regiona, a zatim se poziva ugradjena funkcija koja racuna povrsinu tih labeliranih 
    regiona i koja im nalazi centre jer je potrebno na svakom novcicu na ulaznoj slici ispisati kojoj klasi pripada.
    
    Inputs
    img_in- Ulazna slika novcica
    
    Outputs
    coins_number- Niz od dva elementa prvi je koliko ima novcica od jednog dinara, a drugi koliko ima novica od 5 dinara     
    
    Example
    coin_number = coin_calssification(img_in)
    '''
    #If image is not given as parameter
    if(img_in.size==0):
        print("Morate proslediti sliku, prosledili ste prazan niz!");
        return False;
    if(img_in.shape[0]==0 or img_in.shape[1]==0):
        print("Slika mora biti prosledjena kao matrica koja ima visinu i sirinu razlicitu od nule!")
        return False;
    
    #creating arra of 2 elemnts to store coin number by class
    coins_number = np.zeros(2);
    #for input image creating binary mask
    img_bin = coin_mask(img_in);
    #for binry mask labelin conected components
    bw_img  = bw_label(img_bin);
    #measuring regions area
    regions = skimage.measure.regionprops(bw_img);
    
    areas = [];
    for i,region in zip(range(len(regions)), regions):
            if(i!=0):
                areas.append(region['Area'])
    
    areas.sort();
    dif = 0;
    max_gap = 0;
    treshold = 0;
    for i in range (0,len(areas)-1):
        dif = areas[i+1]-areas[i];
        if(dif>max_gap):
            max_gap=dif;
            treshold = areas[i];
            
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 7), dpi=100);
    
    image_label_overlay = skimage.color.label2rgb(bw_img, image=img_in, bg_label=0);
    ax.imshow(image_label_overlay, cmap='gray'), ax.set_title('Labelirana slika'), ax.axis('off')
    
    for i, region in zip(range(len(regions)), regions):
        if(i!=0):
            val = region['Area'];
            if(val<=treshold):
                coins_number[0] +=1;
                val = 1
            else:
                coins_number[1] +=1;
                val = 5;
            ax.annotate(text=str(val), xy=(region['Centroid'][1], region['Centroid'][0]), xytext=(region['Centroid'][1], region['Centroid'][0]),
                    horizontalalignment='center', verticalalignment='top', fontsize=12)

    imsave('new_coin_count.jpg',image_label_overlay);
    return coins_number;
data_dir = 'sekvence/coins/';


image = skimage.img_as_float((io.imread(os.path.join(data_dir, 'coins1.jpg'))));
coins_number = coin_classification(image);
print(coins_number);

Sve obradjene slike na kojima je na novcicima ispisano kojim klasam pripadaju:


>Slika 1
>>![coins1](sekvence/1count.jpg)

>Slika 2
>>![coins2](sekvence/2count.jpg)

>Slika 3
>>![coins3](sekvence/3count.jpg)

>Slika 4
>>![coins4](sekvence/4count.jpg)

>Slika 5
>>![coins5](sekvence/5count.jpg)

>Slika 6
>>![coins6](sekvence/6count.jpg)

>Slika 7
>>![coins7](sekvence/7count.jpg)

>Slika 8
>>![coins8](sekvence/8count.jpg)

>Slika 9
>>![coins9](sekvence/9count.jpg)